### 1. Configuration de Key vault sur Azure
- Azure Key vault  est un service pour stocker les clés et certficats Azure. 
- Aprés avoir creer sa clé sur Azure , ajoutez #secrets/createScope à l'URL de votre workspace Databricks. 

Example: 
"https://adb-533378876492000.0.azuredatabricks.net/" +  "#secrets/createScope"

### 2. Récupérer un secret avec dbutils

In [0]:
storage_account_key = dbutils.secrets.get(scope="key_adls", key="storage_key")
# Configuration de l'accès au compte de stockage Azure comptestorage0000000001
spark.conf.set("fs.azure.account.key.comptestorage0000000001.dfs.core.windows.net", storage_account_key)

In [0]:
secret=dbutils.secrets.get(scope="key_adls", key="storage_key")
print(secret)

### 💡 Astuce : Afficher une redacted value

In [0]:
clear=""

for c in secret:
    clear+=c+" "
print(clear)

### 🛠️  3. Display un fichier csv situé dans un conteneur abfss 
### 💡  ABFSS : Azure Blob File System Secure – utilisé pour accéder aux fichiers dans un Data Lake Gen2 avec l'authentification sécurisée.

In [0]:

#Chemin vers le conteneur Azure "test"
source_path = "abfss://test@comptestorage0000000001.dfs.core.windows.net/input/"
#fichier situé dans le conteneur
File_name="loto_201902__.csv"

FullURL=source_path+File_name

df= spark.read.format("csv").option("header", "true").option("delimiter", ";").load(FullURL)

display(df)

### 4. 🧪 Cas d’usage:
### Filtre sur le Data frame:
Nous voudrions selectionner les gagnants du loto ayant les conditions suivantes:

- date_de_tirage > 01/01/2019

- date_de_forclusion < 01/01/2020

- jour_de_tirage == "SAMEDI"

In [0]:
from pyspark.sql.functions import col, to_date, sum
from pyspark.sql.types import IntegerType

# Conversion des dates (format français)
df = df.withColumn("date_de_tirage", to_date(col("date_de_tirage"), "dd/MM/yyyy")) \
       .withColumn("date_de_forclusion", to_date(col("date_de_forclusion"), "dd/MM/yyyy"))

df_filtred = df.filter(
    (col("date_de_tirage") > "2019-01-01") &
    (col("date_de_forclusion") < "2020-01-01") &
    (col("jour_de_tirage").isNotNull()) &
    (col("jour_de_tirage").rlike("(?i)^samedi\\s*$"))  # insensible à la casse, ignore les espaces
)

df_filtred = df_filtred.withColumn("nombre_de_gagnant_au_rang1", col("nombre_de_gagnant_au_rang1").cast(IntegerType()))

total_gagnants = df_filtred.select(sum("nombre_de_gagnant_au_rang1")).collect()[0][0]

print("Nombre total de gagnants au rang 1 (tirages du samedi en 2019) :", total_gagnants)

### 5. 📌 Création d'une vue temporaire dans le catalog Databricks

Maintenant, nous voudrions creer une vue temporaire (i.e dans le notebook uniquement).

Nous suiverons le workflow suivant: 
1. Lire le CSV
2. Nettoyage minimal
3. Creer la vue temporaire
4. Interroger la vue via SQL 
5. Sauvegarder le résultat dans une table Delta (dans le catalog Databricks)

In [0]:
from pyspark.sql.functions import to_date, col

# 1. Lire le CSV
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ";")   
    .load(FullURL)
)

# 2. Nettoyage minimal : convertir les colonnes de date et les gagnants en types corrects
df_cleaned = (
    df.withColumn("date_de_tirage", to_date("date_de_tirage", "dd/MM/yyyy"))
      .withColumn("date_de_forclusion", to_date("date_de_forclusion", "dd/MM/yyyy"))
      .withColumn("nombre_de_gagnant_au_rang1", col("nombre_de_gagnant_au_rang1").cast("int"))
)

# 3. Créer une vue temporaire (dans le notebook uniquement)
df_cleaned.createOrReplaceTempView("vue_loto")

# 4. Interroger la vue via SQL (facultatif, par exemple pour vérifier les samedis de 2019)
df_result = spark.sql("""
    SELECT 
        date_de_tirage,
        jour_de_tirage,
        nombre_de_gagnant_au_rang1
    FROM vue_loto
    WHERE 
        date_de_tirage >= '2019-01-01'
        AND date_de_forclusion < '2020-01-01'
        AND UPPER(TRIM(jour_de_tirage)) = 'SAMEDI'
""")

# 5. Sauvegarder le résultat dans une table Delta (dans le catalog Databricks)
df_result.write.format("delta").mode("overwrite").saveAsTable("loto_samedis_2019")

In [0]:
%sql
SELECT * FROM loto_samedis_2019 LIMIT 10
